### Import libraries

In [58]:
import time
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Data Extraction and Pagination Functions

In [59]:
def get_current_page(driver, retries=3):
    """
    Attempts to fetch the current page number.
    Retries a few times in case of a stale element exception.
    """
    for attempt in range(retries):
        try:
            element = driver.find_element(By.CSS_SELECTOR, "a.paginate_button.current")
            # Remove commas from the text before converting to int
            return int(element.text.replace(',', ''))
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(1)
            else:
                raise e

    
def extract_table_data():
    """
    Extracts data rows from the table with id 'myData'.
    """
    data = []
    table = driver.find_element(By.ID, "myData")
    tbody = table.find_element(By.TAG_NAME, "tbody")
    rows = tbody.find_elements(By.TAG_NAME, "tr")
    print(f"Found {len(rows)} rows in tbody.")
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = [cell.text.strip() for cell in cells if cell.text.strip() != ""]
        if row_data:
            data.append(row_data)
    return data

### Selenium Data Extraction Script

In [62]:
# Initialize the WebDriver (Chrome in this example)
driver = webdriver.Chrome()
driver.get("https://medapps.sahpra.org.za:6006/")

# Create an explicit wait object
wait = WebDriverWait(driver, 20)

# Wait until the table with id "myData" is present and its <tbody> has at least one row.
wait.until(lambda d: len(d.find_element(By.ID, "myData")
                            .find_element(By.TAG_NAME, "tbody")
                            .find_elements(By.TAG_NAME, "tr")) > 0)

all_data = []

# Extract header row from the table's <thead>
table = driver.find_element(By.ID, "myData")
thead = table.find_element(By.TAG_NAME, "thead")
headers = [th.text.strip() for th in thead.find_elements(By.TAG_NAME, "th")]
all_data.append(headers)

page = 1

while True:
    print(f"\nExtracting data from page {page}...")
    page_rows = extract_table_data()
    print(f"Page {page}: {len(page_rows)} data rows found.")
    all_data.extend(page_rows)
    
    # Locate the Next button using its id "myData_next"
    try:
        next_btn = driver.find_element(By.ID, "myData_next")
    except Exception as e:
        print("Next button not found:", e)
        break

    # Check if the Next button is disabled (DataTables adds a "disabled" class when no further pages exist)
    if "disabled" in next_btn.get_attribute("class"):
        print("Next button is disabled. Reached the last page.")
        break

    # Get the current page number before clicking
    try:
        current_page = get_current_page(driver)
    except Exception as e:
        print("Failed to get current page:", e)
        break
    print(f"Current page before click: {current_page}")
    
    # Click the Next button
    next_btn.click()
    
    # Wait until the page number increases, with a timeout of 60 seconds
    try:
        WebDriverWait(driver, 60).until(lambda d: get_current_page(d) > current_page)
    except Exception as e:
        print("Timed out waiting for page update:", e)
        break

    page += 1

driver.quit()


Extracting data from page 1...
Found 10 rows in tbody.
Page 1: 10 data rows found.
Current page before click: 1

Extracting data from page 2...
Found 10 rows in tbody.
Page 2: 10 data rows found.
Current page before click: 2

Extracting data from page 3...
Found 10 rows in tbody.
Page 3: 10 data rows found.
Current page before click: 3

Extracting data from page 4...
Found 10 rows in tbody.
Page 4: 10 data rows found.
Current page before click: 4

Extracting data from page 5...
Found 10 rows in tbody.
Page 5: 10 data rows found.
Current page before click: 5

Extracting data from page 6...
Found 10 rows in tbody.
Page 6: 10 data rows found.
Current page before click: 6

Extracting data from page 7...
Found 10 rows in tbody.
Page 7: 10 data rows found.
Current page before click: 7

Extracting data from page 8...
Found 10 rows in tbody.
Page 8: 10 data rows found.
Current page before click: 8

Extracting data from page 9...
Found 10 rows in tbody.
Page 9: 10 data rows found.
Current page

### Save the collected data into a CSV file using pandas.

In [63]:
df = pd.DataFrame(all_data[1:], columns=all_data[0])
df.to_csv("sahpra-health-products-data.csv", index=False)
print(f"\nScraping complete. Data from {page} pages saved to sahpra_data.csv.")


Scraping complete. Data from 2007 pages saved to sahpra_data.csv.
